In [1]:
from pyspark.sql import SparkSession
# import pymysql

spark = SparkSession.builder.appName("JDBC Connection").config("spark.driver.extraClassPath", "/Users/qingyanchen/Downloads/mysql-connector-java-8.0.25.jar").getOrCreate()

# 设置 JDBC 配置信息
jdbc_url = "jdbc:mysql://localhost:3306/testdata"
jdbc_driver = "com.mysql.jdbc.Driver"
jdbc_username = "root"
jdbc_password = "123456"

# 创建 DataFrame
df = spark.read.jdbc(url=jdbc_url, table="mytable", properties={"driver": jdbc_driver, "user": jdbc_username, "password": jdbc_password})


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/22 10:39:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/22 10:39:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.


In [2]:
df.show()

+---+----+-----+
| id|name|count|
+---+----+-----+
|  1|John|    1|
|  2|Jane|    1|
|  3|Mary|    1|
|  4| Tom|    1|
+---+----+-----+



In [3]:
# 导入必要的模块
from pyspark.sql.functions import count

# 创建一个DataFrame
df = spark.createDataFrame([(1, "John", 25), (2, "Jane", 30), (3, "Mary", 35), (4, "Tom", 40)], ["id", "name", "age"])

# 对第一列进行分组，然后对第二列进行分组
result = df.groupBy("id", "name").agg(count("*").alias("count"))

# 打印结果
result.show()

+---+----+-----+
| id|name|count|
+---+----+-----+
|  1|John|    1|
|  2|Jane|    1|
|  3|Mary|    1|
|  4| Tom|    1|
+---+----+-----+



In [4]:
#读取日志文件
web_logs = spark.read.json('web_log1.json')
#选择所要的字段
time_ip=web_logs.select("timestamp","ip_address")
time_ip.show()

+-------------------+---------------+
|          timestamp|     ip_address|
+-------------------+---------------+
|2023-02-25 09:29:52| 118.144.72.252|
|2023-01-11 20:24:30| 123.100.190.59|
|2022-07-02 13:49:32|123.199.128.174|
|2023-01-05 13:06:53| 210.185.192.55|
|2022-08-31 12:32:08|  124.68.89.156|
|2022-08-14 20:50:43|  119.27.64.233|
|2022-07-24 11:33:44|   119.48.4.102|
|2022-08-14 18:56:40| 117.120.64.237|
|2023-02-11 04:47:49|123.101.118.142|
|2022-07-25 22:23:42| 218.192.230.43|
|2023-03-19 20:24:51|  119.96.54.177|
|2023-04-17 23:44:47| 116.214.64.170|
|2022-06-22 09:20:29| 202.127.16.140|
|2022-12-30 07:08:46| 210.16.128.254|
|2022-08-29 12:52:37|   219.82.0.175|
|2022-12-08 16:05:38| 210.51.207.198|
|2023-03-21 20:26:57|  203.91.32.151|
|2023-01-19 10:57:05| 202.125.176.44|
|2022-10-01 15:17:19|  221.129.67.57|
|2023-03-08 05:27:33|  203.88.32.137|
+-------------------+---------------+
only showing top 20 rows



In [7]:
# Define the JDBC URL for the MySQL database
url = "jdbc:mysql://localhost:3306/testdata"

# Define the properties for the JDBC driver
properties = {
    "user": "root",
    "password": "123456",
    "driver": "com.mysql.jdbc.Driver"
}

# Write the data to the MySQL database
time_ip.write \
    .mode("append") \
    .jdbc(url=url, table="timeip", properties=properties)

In [9]:
df = spark.read.jdbc(url=jdbc_url, table="mytable", properties={"driver": jdbc_driver, "user": jdbc_username, "password": jdbc_password})
df.show()


+---+----+-----+
| id|name|count|
+---+----+-----+
|  1|John|    1|
|  2|Jane|    1|
|  3|Mary|    1|
|  4| Tom|    1|
+---+----+-----+



In [2]:
from pyspark.sql.functions import col
from pyspark.sql import functions
from pyspark.sql import SparkSession
# import pymysql

spark = SparkSession.builder.appName("JDBC Connection").config("spark.driver.extraClassPath", "/Users/qingyanchen/Downloads/mysql-connector-java-8.0.25.jar").getOrCreate()

web_logs = spark.read.json('web_log1.json')
web_logs2=web_logs.select("timestamp","response_content_length")
split_col = functions.split(web_logs2['timestamp'], ' ')
web_logs2=web_logs2.withColumn('timestamp', split_col.getItem(0))

web_logs2=web_logs2.withColumn('response_content_length',functions.split(web_logs2['response_content_length'],'bytes').getItem(0))

web_logs2=web_logs2.withColumn('response_content_length',col('response_content_length').cast("int"))
web_logs2=web_logs2.groupby("timestamp").sum("response_content_length").alias("response_content_length")
web_logs2=web_logs2.select(web_logs2["timestamp"],web_logs2["sum(response_content_length)"].alias("response_content_length"))
url = "jdbc:mysql://localhost:3306/testdata"

# Define the properties for the JDBC driver
properties = {
    "user": "root",
    "password": "123456",
    "driver": "com.mysql.jdbc.Driver"
}
web_logs2.write.mode("append").jdbc(url=url, table="time_sum", properties=properties)
web_logs2.show()

Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.


+----------+-----------------------+
| timestamp|response_content_length|
+----------+-----------------------+
|2022-10-05|              152077609|
|2023-01-21|              120910569|
|2023-05-01|              129253547|
|2023-04-17|              128518167|
|2023-04-21|              131877678|
|2023-04-28|              137402571|
|2022-10-07|              146889317|
|2023-02-10|              129507120|
|2022-05-17|              137581041|
|2023-05-04|              125880810|
|2023-04-26|              130263786|
|2023-03-10|              117255798|
|2023-03-11|              108219484|
|2022-07-04|              136946829|
|2023-02-23|              137311455|
|2022-07-08|              120125752|
|2022-07-30|              128714011|
|2023-03-17|              123034522|
|2022-09-03|              127935542|
|2022-07-23|              149043693|
+----------+-----------------------+
only showing top 20 rows



In [45]:
from pyspark.sql.functions import col
from pyspark.sql import functions
from pyspark.sql import SparkSession
# import pymysql

spark = SparkSession.builder.appName("JDBC Connection").config("spark.driver.extraClassPath", "/Users/qingyanchen/Downloads/mysql-connector-java-8.0.25.jar").getOrCreate()

web_logs = spark.read.json('web_log1.json')
web_logs2=web_logs.select("timestamp","response_content_length")
split_col = functions.split(web_logs2['timestamp'], ' ')
web_logs2=web_logs2.withColumn('year', functions.split(web_logs2['timestamp'], '-').getItem(0))
web_logs2=web_logs2.withColumn('month', functions.split(web_logs2['timestamp'], '-').getItem(1))
web_logs2=web_logs2.withColumn('day', functions.split(web_logs2['timestamp'], '-').getItem(1))

web_logs2=web_logs2.withColumn('time', functions.split(web_logs2['timestamp'], ' ').getItem(1))
web_logs2=web_logs2.withColumn('response_content_length',functions.split(web_logs2['response_content_length'],'bytes').getItem(0)).orderBy("timestamp")
web_logs2.show()


web_logs3=web_logs2.withColumn('timestamp',functions.concat_ws('-',web_logs2['year'],web_logs2['month'])).select("timestamp","response_content_length")
web_logs3=web_logs3.withColumn('response_content_length',col('response_content_length').cast("int"))
web_logs3=web_logs3.groupby("timestamp").count().alias("count").orderBy("timestamp")
# web_logs3.write.mode("append").jdbc(url=url, table="time_sum", properties=properties)
# web_logs3=web_logs3.select(web_logs3["timestamp"],web_logs3["sum(response_content_length)"].alias("response_content_length")).orderBy("timestamp")
web_logs3.show()
web_logs2.write.mode("append").jdbc(url=url, table="time_count", properties=properties)


web_logs2=web_logs2.withColumn('response_content_length',col('response_content_length').cast("int"))
web_logs2=web_logs2.groupby("timestamp").sum("response_content_length").alias("response_content_length")
web_logs2=web_logs2.select(web_logs2["timestamp"],web_logs2["sum(response_content_length)"].alias("response_content_length"))
# web_logs3.write.mode("append").jdbc(url=url, table="time_count", properties=properties)
web_logs2.show()

+-------------------+-----------------------+----+-----+---+--------+
|          timestamp|response_content_length|year|month|day|    time|
+-------------------+-----------------------+----+-----+---+--------+
|2020-05-01 00:14:45|                  67616|2020|   05| 05|00:14:45|
|2020-05-01 00:17:59|                 884584|2020|   05| 05|00:17:59|
|2020-05-01 00:22:54|                 495177|2020|   05| 05|00:22:54|
|2020-05-01 00:28:33|                 217380|2020|   05| 05|00:28:33|
|2020-05-01 00:59:05|                  24874|2020|   05| 05|00:59:05|
|2020-05-01 01:05:52|                 769806|2020|   05| 05|01:05:52|
|2020-05-01 01:23:38|                 190584|2020|   05| 05|01:23:38|
|2020-05-01 01:24:12|                  67674|2020|   05| 05|01:24:12|
|2020-05-01 02:09:27|                 412041|2020|   05| 05|02:09:27|
|2020-05-01 02:32:30|                 735547|2020|   05| 05|02:32:30|
|2020-05-01 02:52:48|                 192387|2020|   05| 05|02:52:48|
|2020-05-01 03:09:08

+-------------------+-----------------------+
|          timestamp|response_content_length|
+-------------------+-----------------------+
|2023-04-30 19:21:04|                 757957|
|2022-10-05 22:43:32|                 980446|
|2022-08-15 08:50:38|                 376207|
|2022-12-29 18:30:47|                 487427|
|2023-03-15 12:56:29|                 614381|
|2022-05-25 04:03:05|                 326598|
|2022-09-08 17:32:54|                 225532|
|2022-08-26 17:18:39|                 531206|
|2022-05-28 12:27:37|                  93236|
|2023-04-27 13:00:43|                 580012|
|2022-05-27 01:35:19|                 616888|
|2023-01-13 01:53:44|                 696999|
|2022-10-21 09:43:01|                 225673|
|2022-08-09 19:45:15|                 940934|
|2022-06-16 07:05:28|                 564542|
|2023-04-28 18:43:38|                 305806|
|2022-10-10 13:04:59|                 579152|
|2023-01-03 15:11:24|                  93368|
|2022-08-09 08:13:30|             

In [10]:
from pyspark.sql import functions
# 读取JSON格式的Web日志文件
# 
web_logs = spark.read.json('web_log1.json')
web_logs2=web_logs.select("timestamp","response_content_length")
split_col = functions.split(web_logs2['timestamp'], ' ')
web_logs2=web_logs2.withColumn('timestamp', split_col.getItem(1))
web_logs2.show()
web_logs2=web_logs2.withColumn('response_content_length',functions.split(web_logs2['response_content_length'],'bytes').getItem(0))
web_logs2.show()
from pyspark.sql.functions import col
# df = df.withColumn("age", col("age").cast("float"))
web_logs2=web_logs2.withColumn('response_content_length',col('response_content_length').cast("int"))
web_logs2=web_logs2.groupby("timestamp").sum("response_content_length")
# select count() from mydata.mytable
web_logs2.show()


+---------+-----------------------+
|timestamp|response_content_length|
+---------+-----------------------+
| 09:29:52|            305864bytes|
| 20:24:30|             50292bytes|
| 13:49:32|            900609bytes|
| 13:06:53|            542058bytes|
| 12:32:08|             32101bytes|
| 20:50:43|             17540bytes|
| 11:33:44|             54501bytes|
| 18:56:40|            870018bytes|
| 04:47:49|            456133bytes|
| 22:23:42|             42329bytes|
| 20:24:51|            855881bytes|
| 23:44:47|            813620bytes|
| 09:20:29|             17861bytes|
| 07:08:46|            623334bytes|
| 12:52:37|            395794bytes|
| 16:05:38|            119055bytes|
| 20:26:57|            252035bytes|
| 10:57:05|            149316bytes|
| 15:17:19|            128064bytes|
| 05:27:33|            662544bytes|
+---------+-----------------------+
only showing top 20 rows

+---------+-----------------------+
|timestamp|response_content_length|
+---------+-----------------------+
| 

In [13]:
from pyspark.sql import functions
# 读取JSON格式的Web日志文件
# 
web_logs = spark.read.json('web_log1.json')
web_logs2=web_logs.select("timestamp","response_content_length")
split_col = functions.split(web_logs2['timestamp'], ' ')
web_logs2=web_logs2.withColumn('timestamp', split_col.getItem(1))
web_logs2.show()
web_logs2=web_logs2.withColumn('response_content_length',functions.split(web_logs2['response_content_length'],'bytes').getItem(0))
web_logs2.show()
from pyspark.sql.functions import col
# df = df.withColumn("age", col("age").cast("float"))
web_logs2=web_logs2.withColumn('response_content_length',col('response_content_length').cast("int"))
web_logs2=web_logs2.groupby("timestamp").sum("response_content_length")
# select count() from mydata.mytable
web_logs2.show()


+---------+-----------------------+
|timestamp|response_content_length|
+---------+-----------------------+
| 09:29:52|            305864bytes|
| 20:24:30|             50292bytes|
| 13:49:32|            900609bytes|
| 13:06:53|            542058bytes|
| 12:32:08|             32101bytes|
| 20:50:43|             17540bytes|
| 11:33:44|             54501bytes|
| 18:56:40|            870018bytes|
| 04:47:49|            456133bytes|
| 22:23:42|             42329bytes|
| 20:24:51|            855881bytes|
| 23:44:47|            813620bytes|
| 09:20:29|             17861bytes|
| 07:08:46|            623334bytes|
| 12:52:37|            395794bytes|
| 16:05:38|            119055bytes|
| 20:26:57|            252035bytes|
| 10:57:05|            149316bytes|
| 15:17:19|            128064bytes|
| 05:27:33|            662544bytes|
+---------+-----------------------+
only showing top 20 rows

+---------+-----------------------+
|timestamp|response_content_length|
+---------+-----------------------+
| 

In [ ]:
from pyspark.sql import functions
# 读取JSON格式的Web日志文件
# 
web_logs = spark.read.json('web_log1.json')
web_logs2=web_logs.select("timestamp","response_content_length")
split_col = functions.split(web_logs2['timestamp'], ' ')
web_logs2=web_logs2.withColumn('timestamp', split_col.getItem(1))
web_logs2.show()
web_logs2=web_logs2.withColumn('response_content_length',functions.split(web_logs2['response_content_length'],'bytes').getItem(0))
web_logs2.show()
from pyspark.sql.functions import col
# df = df.withColumn("age", col("age").cast("float"))
web_logs2=web_logs2.withColumn('response_content_length',col('response_content_length').cast("int"))
web_logs2=web_logs2.groupby("timestamp").sum("response_content_length")
# select count() from mydata.mytable
web_logs2.show()


In [6]:
from pyspark.sql import functions
from pyspark.sql.functions import col,count
from pyspark.sql import SparkSession
# import pymysql

spark = SparkSession.builder.appName("JDBC Connection").config("spark.driver.extraClassPath", "/Users/qingyanchen/Downloads/mysql-connector-java-8.0.25.jar").getOrCreate()

web_logs = spark.read.json('web_log1.json')
load_type=web_logs.select("timestamp","load_type")
#生成对类型切割的函数
split_type=functions.split(load_type["load_type"],'/')
#对类型列进行切割
load_type2=load_type.withColumn('load_type',split_type.getItem(0))
#生成对时间戳切割的函数
load_type3 = functions.split(load_type2['timestamp'], ' ')
#对时间戳列进行切割
load_type4=load_type2.withColumn('timestamp', load_type3.getItem(0))
load_type4=load_type4.groupBy("timestamp","load_type").agg(count("*").alias("count"))
# load_type4=load_type4
load_type4.show()
# Define the JDBC URL for the MySQL database
url = "jdbc:mysql://localhost:3306/testdata"
# Define the properties for the JDBC driver
properties = {
    "user": "root",
    "password": "123456",
    "driver": "com.mysql.jdbc.Driver"
}
# load_type4.write.mode("append").jdbc(url=url, table="type_count", properties=properties)
load_type4.write.mode("append").jdbc(url=url, table="type_count", properties=properties)


+----------+---------+-----+
| timestamp|load_type|count|
+----------+---------+-----+
|2022-07-10|     党政|   27|
|2022-05-01|     观点|   24|
|2022-05-17|     地方|  173|
|2023-02-21|     观点|   27|
|2022-08-30|     党政|   36|
|2022-10-08|     党政|   25|
|2022-10-12|     要闻|   30|
|2022-10-28|     要闻|   43|
|2023-01-19|     地方|  183|
|2023-03-16|     观点|   29|
|2022-11-30|     要闻|   46|
|2023-04-20|     观点|   32|
|2023-05-03|     党政|   34|
|2023-04-10|     党政|   26|
|2022-10-29|     观点|   24|
|2022-10-10|     要闻|   43|
|2022-05-26|     观点|   31|
|2022-12-28|     党政|   34|
|2022-07-13|     党政|   20|
|2022-05-05|     要闻|   50|
+----------+---------+-----+
only showing top 20 rows



In [47]:
# Define the JDBC URL for the MySQL database
url = "jdbc:mysql://localhost:3306/testdata"

# Define the properties for the JDBC driver
properties = {
    "user": "root",
    "password": "123456",
    "driver": "com.mysql.jdbc.Driver"
}
load_type4.write.mode("append").jdbc(url=url, table="type_count", properties=properties)
load_type4.show()

+----------+---------+-----+
| timestamp|load_type|count|
+----------+---------+-----+
|2023-05-03|     党政|   30|
|2023-05-02|     观点|   26|
|2023-05-08|     观点|   18|
|2023-05-01|     观点|   16|
|2023-05-09|     观点|   20|
|2023-05-07|     党政|   16|
|2023-05-05|     观点|   17|
|2023-05-05|     地方|  151|
|2023-05-02|     党政|   35|
|2023-05-09|     要闻|   31|
|2023-05-01|     地方|  152|
|2023-05-05|     党政|   35|
|2023-05-04|     地方|  150|
|2023-05-06|     地方|  159|
|2023-05-08|     地方|  151|
|2023-05-03|     观点|   20|
|2023-05-06|     观点|   15|
|2023-05-03|     要闻|   34|
|2023-05-08|     党政|   25|
|2023-05-09|     党政|   31|
+----------+---------+-----+
only showing top 20 rows



In [16]:
# split_col = functions.split(web_logs['timestamp'], ' ')
# web_logs.withColumn('timestamp', split_col.getItem(1)).show(2)
# df = web_logs.withColumn("timestamp", to_timestamp(col("timestamp"), "yyyy-MM-dd HH:mm:ss"))
start_time = "2023-05-04 00:00:00"
end_time = "2023-05-05 23:59:59"
# web_logs.agg(count("*").alias("count")).show()
web_logs2=web_logs.filter((col("timestamp") >= start_time) & (col("timestamp") <= end_time))
split_col = functions.split(web_logs2['timestamp'], ':')
web_logs2=web_logs2.withColumn('timestamp', split_col.getItem(0))
web_logs2.groupBy("timestamp").agg(count("*").alias("count")).show()

NameError: name 'count' is not defined

In [20]:
start_time = "2023-05-03 00:00:00"
end_time = "2023-05-04 23:59:59"
# web_logs.agg(count("*").alias("count")).show()
web_logs.filter((col("timestamp") >= start_time) & (col("timestamp") <= end_time)).agg(count("*").alias("count")).show()

+-----+
|count|
+-----+
|  361|
+-----+



In [13]:
from pyspark.sql import functions
from pyspark.sql.functions import col,count
from pyspark.sql import SparkSession
# import pymysql

spark = SparkSession.builder.appName("JDBC Connection").config("spark.driver.extraClassPath", "/Users/qingyanchen/Downloads/mysql-connector-java-8.0.25.jar").getOrCreate()


web_logs = spark.read.json('web_log.json')
web_logs=web_logs.select("timestamp","load_type")

web_logs2=web_logs.filter((col("load_type").like("地方%")))
# web_logs2.show()
split_col = functions.split(web_logs2['load_type'], '/')
web_logs2=web_logs2.withColumn('load_type', split_col.getItem(1))
#生成对时间戳切割的函数
load_type3 = functions.split(web_logs2['timestamp'], ' ')
#对时间戳列进行切割
web_logs2=web_logs2.withColumn('timestamp', load_type3.getItem(0))
web_logs2=web_logs2.groupBy("timestamp","load_type").agg(count("*")).alias("count")
web_logs2.show()
# Define the JDBC URL for the MySQL database
url = "jdbc:mysql://localhost:3306/testdata"

# Define the properties for the JDBC driver
properties = {
    "user": "root",
    "password": "123456",
    "driver": "com.mysql.jdbc.Driver"
}
web_logs2.write.mode("append").jdbc(url=url, table="difang_sum", properties=properties)

+----------+---------+--------+
| timestamp|load_type|count(1)|
+----------+---------+--------+
|2015-03-18|   内蒙古|       1|
|2021-05-25|     浙江|       3|
|2004-09-12|     贵州|       1|
|2014-01-25|     云南|       1|
|2006-03-08|     江苏|       3|
|2022-12-29|     宁夏|       4|
|2013-04-14|     陕西|       1|
|2002-11-06|     新疆|       1|
|2004-05-07|     海南|       1|
|2015-04-21|     辽宁|       2|
|2005-10-20|     湖南|       1|
|2022-04-15|     吉林|       1|
|2017-01-31|     西藏|       1|
|2019-12-16|     吉林|       1|
|2002-05-12|     山西|       2|
|2007-05-27|     安徽|       1|
|2004-07-12|     山东|       1|
|2019-07-07|     云南|       1|
|2015-12-04|     四川|       1|
|2021-07-02|     重庆|       3|
+----------+---------+--------+
only showing top 20 rows



In [9]:
# -*- coding: utf-8 -*-
"""
@File    : ip_util.py
@Date    : 2022-10-13
"""
import requests


def get_ip_info(ip):
    url = 'http://ip.taobao.com/outGetIpInfo'

    params = {
        'ip': ip,
        'accessKey': 'alibaba-inc'
    }
    
    res = requests.get(url, params)

    if not res.ok:
        res.raise_for_status()

    return res.json().get('data')


if __name__ == '__main__':
    
    for i in ['221.218.209.125','202.91.128.133']:
        print(i)
    
    print(get_ip_info('221.218.209.125'))
    print(get_ip_info('202.91.128.133'))


221.218.209.125
202.91.128.133
{'area': '', 'country': '中国', 'isp_id': '100026', 'queryIp': '221.218.209.125', 'city': '北京', 'ip': '221.218.209.125', 'isp': '联通', 'county': '', 'region_id': '110000', 'area_id': '', 'county_id': None, 'region': '北京', 'country_id': 'CN', 'city_id': '110100'}


HTTPError: 404 Client Error: Not Found for url: https://ip.taobao.com/outGetIpInfo?ip=202.91.128.133&accessKey=alibaba-inc

In [3]:
# Author:52loli
# Time:2020.10.06
import requests
import json
 
class Ip():
    def __init__(self):
        self.ip_url = 'https://www.toolnb.com/Tools/Api/IP.html'
        self.ipInfo_url = 'https://www.toolnb.com/Tools/Api/ipgetareainfo.html'
        self.headers = {
            'cookie':'PHPSESSID=q8e8vhzg8nsarisdwsbmxb1g72',
            'referer':'https://www.toolnb.com/tools/ipgetareainfo.html',
            'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.4183.121 Safari/537.36'
        }
    def get_ip(self):
        tmp = requests.post(self.ip_url,headers=self.headers)
        result = json.loads(tmp.text)
        return result['data']['ip']
    def get_IPaddress(self):
        data = {'ip': self.get_ip()}
        tmp = requests.post(self.ipInfo_url,data = data,headers=self.headers)
        res = json.loads(tmp.text)
        print(res['data']['area'])
 
ip = Ip()
ip.get_IPaddress()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [27]:
import geoip2.database

# 加载GeoIP2数据库
reader = geoip2.database.Reader('GeoLite2-City.mmdb')

# 要查询的IP地址
ip_address = '210.185.192.55'

try:
    # 查询IP地址的地理位置信息
    response = reader.city(ip_address)

    # 提取地理位置信息
    country = response.country.name
    city = response.city.name
    latitude = response.location.latitude
    longitude = response.location.longitude

    # 打印结果
    print(f'IP地址: {ip_address}')
    print(f'国家: {country}')
    print(f'城市: {city}')
    print(f'纬度: {latitude}')
    print(f'经度: {longitude}')

except geoip2.errors.AddressNotFoundError:
    print(f'找不到IP地址: {ip_address}')

finally:
    # 关闭GeoIP2数据库连接
    reader.close()


IP地址: 210.185.192.55
国家: China
城市: None
纬度: 34.7732
经度: 113.722
